In [1]:
%load_ext autoreload
%autoreload 2
import warnings
import numpy as np
import time
import os
from dotenv import load_dotenv
os.environ["USE_PYGEOS"] = "0"
import napari_sparrow as nas
from spatialdata import read_zarr
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scanpy as sc
%run own_and_modified_functions.ipynb
warnings.simplefilter(action='ignore')

the value of the environment variable BASIC_DCT_BACKEND is not in ["JAX","SCIPY"]


In [2]:
OUTPUT_DIR = '/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data'
sdata = read_zarr( os.path.join(  OUTPUT_DIR, 'sdata.zarr'  ) )
anndata = sc.read('/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/anndataNucA11') # reset original anndata
# delete anndata
del sdata.table
# create anndata
sdata.table = anndata

In [3]:
n_neighbors = 15
n_PCAs = 23
cluster_resolution = 1
# make umap and do leiden clustering with scanpy functions
make_umap(sdata,n_neighbors=n_neighbors,n_PCAs=n_PCAs)
column_name = 'leiden_'+str(n_PCAs)+'_'+str(n_neighbors)+'_'+str(cluster_resolution)
sc.tl.leiden(sdata.table,resolution=cluster_resolution,key_added=column_name)

In [4]:

# annotate each leiden cluster
# method based on marker genes and similar to 'own_score_genes' but leiden clusters annotated instead of individual cells
# for each leiden cluster, a score is calculated for each cell type: 
# sum of the mean expressions of the markers in leiden cluster - sum of mean expression of the markers in all cells
n_clusters = np.unique(sdata.table.obs[column_name]).size
leiden_mean_expression = {}
for i in range(n_clusters):
    an_cluster = sdata.table[sdata.table.obs[column_name]==str(i)]
    daf = an_cluster.to_df().mean(axis=0)
    pd.DataFrame(daf)
    leiden_mean_expression[i] = daf
leiden_mean_expression[0]


gene
Acta2      -0.114241
Adamtsl2   -0.148784
Adgre1     -0.175207
Adgrg6     -0.054919
Atp6v0d2   -0.183765
              ...   
Vwf        -0.141544
Wnt2        0.783007
Wnt9b      -0.010520
Wt1        -0.053019
Xcr1       -0.021488
Length: 99, dtype: float64

In [5]:
norm_expr_var = True
if norm_expr_var:
    all_mean_expression = sdata.table.to_df().div(sdata.table.to_df().std(axis=0)).mean(axis=0)
    for i in range(n_clusters):
        leiden_mean_expression[i] = leiden_mean_expression[i].div(sdata.table.to_df().std(axis=0))
else:
    all_mean_expression = sdata.table.to_df().mean(axis=0)

In [22]:
path_marker_genes = "/home/wout/Documents/Thesis_lokaal/Mouse_Liver_Resolve_Data/markerGeneListBasic.csv",
marker_genes = pd.read_csv(path_marker_genes[0], sep=',',index_col=0)
scores_leiden_celltype = pd.DataFrame()
cell_types = marker_genes.columns.tolist()
for cell_type in cell_types:
    scores_clusters = []
    for i in range(n_clusters):
        score = 0 
        for gene in marker_genes[marker_genes[cell_type] == 1].index.tolist():
            score = score + (leiden_mean_expression[i][gene] - all_mean_expression[gene])
        scores_clusters.append(score)
    scores_leiden_celltype[cell_type] = scores_clusters
scores_leiden_celltype


,Hepa,LSEC,Endo vein,Chol,Kupf,Stel,Fibr,B cell,Meso
0,-3.460093,10.989380,2.278675,-1.790231,-1.301024,-1.822716,-2.907918,-0.432275,-0.346116
1,-0.427371,-2.828320,-1.615156,-0.586995,-3.264427,-3.568880,-3.422267,-0.482294,-0.268901
2,3.792530,-2.820380,-1.384843,0.940080,-2.826036,-2.437137,-2.427748,-0.469303,-0.219206
3,5.111039,-2.846000,-1.362488,1.481186,-2.325098,-0.055361,-0.261462,-0.489113,-0.334417
4,3.773936,-2.858302,-1.444598,0.878113,-2.595199,-2.266554,-2.597574,-0.482150,-0.369402
5,-4.395898,0.292705,-1.147669,-2.254421,30.171808,-1.946341,-2.452922,-0.338389,-0.329075
6,-4.085902,-1.906594,0.063132,-2.263773,-0.245806,21.623266,14.458159,-0.464492,-0.151628
7,3.256718,-2.846059,-1.436028,1.140859,-2.753880,0.978454,-1.842269,-0.486840,-0.216365
8,0.740560,-2.911453,-1.582877,0.224099,-3.308103,-1.099450,-1.217452,-0.490748,-0.177561
9,2.346745,-2.784031,-1.539472,0.351591,-2.955053,-2.177873,-2.277228,-0.491401,-0.195563


In [29]:
# scale scores per cell type (min-max scaler), to make them comparable between cell types 
# min max scale columns of scores_leiden_celltype
scores_leiden_celltype = scores_leiden_celltype.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
# leiden cluster is annotated with the cell type with the highest score
# select columns with values above max of the row - 0.05
scores_leiden_celltype
# round to 6 decimals
scores_leiden_celltype = scores_leiden_celltype.round(6)
scores_leiden_celltype


,Hepa,LSEC,Endo vein,Chol,Kupf,Stel,Fibr,B cell,Meso
0,0.298690,0.427510,0.103279,0.041213,0.066885,0.111843,0.047357,0.003002,0.010811
1,0.546834,0.002932,0.002155,0.100921,0.008674,0.045696,0.031766,0.000550,0.027095
2,0.892117,0.003176,0.008136,0.176699,0.021672,0.088568,0.061912,0.001187,0.037575
3,1.000000,0.002389,0.008716,0.203550,0.036523,0.178792,0.127577,0.000216,0.013278
4,0.890595,0.002011,0.006584,0.173624,0.028515,0.095030,0.056764,0.000557,0.005900
5,0.222120,0.098832,0.014295,0.018179,1.000000,0.107160,0.061149,0.007603,0.014405
6,0.247485,0.031254,0.045740,0.017715,0.098171,1.000000,0.573760,0.001423,0.051827
7,0.848275,0.002387,0.006806,0.186662,0.023811,0.217954,0.079659,0.000327,0.038174
8,0.642397,0.000378,0.002993,0.141170,0.007379,0.139241,0.098599,0.000136,0.046358
9,0.773819,0.004293,0.004120,0.147497,0.017846,0.098389,0.066475,0.000104,0.042562


In [25]:
max_scores = scores_leiden_celltype.max(axis=1)
max_scores

0     0.427510
1     0.546834
2     0.892117
3     1.000000
4     0.890595
5     1.000000
6     1.000000
7     0.848275
8     0.642397
9     0.773819
10    0.934043
11    1.000000
12    0.810756
13    0.302845
14    0.853678
15    1.000000
16    0.578438
17    1.000000
18    0.603795
19    1.000000
20    0.478844
21    0.403851
22    0.062464
dtype: float64

In [31]:
# select for each row of scores_leiden_celltype the columns with values
sc_leiden_cellt = {}
# iterate over rows of scores_leiden_celltype
for i in range(scores_leiden_celltype.shape[0]):
    # select columns with values above max of the row - 0.1
    sc_leiden_cellt[i] = scores_leiden_celltype.columns[scores_leiden_celltype.iloc[i,:] > max_scores[i]-0.1].tolist()
sc_leiden_cellt






{0: ['LSEC'],
 1: ['Hepa'],
 2: ['Hepa'],
 3: ['Hepa'],
 4: ['Hepa'],
 5: ['Kupf'],
 6: ['Stel'],
 7: ['Hepa'],
 8: ['Hepa'],
 9: ['Hepa'],
 10: ['Hepa'],
 11: ['Fibr', 'Meso'],
 12: ['Hepa'],
 13: ['Kupf', 'Fibr', 'B cell', 'Meso'],
 14: ['Hepa'],
 15: ['Chol'],
 16: ['Endo vein'],
 17: ['B cell'],
 18: ['Hepa'],
 19: ['LSEC', 'Endo vein'],
 20: ['LSEC'],
 21: ['Fibr'],
 22: ['Hepa',
  'LSEC',
  'Endo vein',
  'Chol',
  'Kupf',
  'Stel',
  'Fibr',
  'B cell',
  'Meso']}

In [ ]:
sc_leiden_cellt = scores_leiden_celltype.idxmax(axis=1).to_dict()
unknown_cells = scores_leiden_celltype[scores_leiden_celltype.max(axis=1) < 0].index.tolist()
# change the values of keys in list
for i in unknown_cells:
    sc_leiden_cellt[i] = 'unknown'
sc_leiden_cellt = {str(k): v for k, v in sc_leiden_cellt.items()}
sdata.table.obs["annotation_"+column_name]=sdata.table.obs[column_name] 
sdata.table.obs["annotation_"+column_name].replace(list(sc_leiden_cellt.keys()),list(sc_leiden_cellt.values()), inplace=True)
b = pd.DataFrame.from_dict(sc_leiden_cellt, orient='index')
cell_type_leiden = {}
cell_types = np.unique(b[0])
for cell_type in cell_types:
    indices = b.index[b[0] == cell_type].tolist()
    cell_type_leiden[cell_type] = indices
sdata.table.uns["mapping_cell_type_"+column_name] = cell_type_leiden
# cleanliness of the annotation based on highest and second highest score
max_scores = scores_leiden_celltype.max(axis=1)
second_scores = scores_leiden_celltype.apply(lambda x: x.nlargest(2).values[-1], axis=1)
cleanliness = (max_scores - second_scores) / ((abs(max_scores) + abs(second_scores)) / 2)
sc_leiden_cleanl = cleanliness.to_dict()
for i in unknown_cells:
    sc_leiden_cleanl[i] = 0
sc_leiden_cleanl = {str(k): v for k, v in sc_leiden_cleanl.items()}
sdata.table.obs["cleanliness_"+column_name]=sdata.table.obs[column_name] 
sdata.table.obs["cleanliness_"+column_name].replace(list(sc_leiden_cleanl.keys()),list(sc_leiden_cleanl.values()), inplace=True)

In [13]:
scores_leiden_celltype 

,Hepa,LSEC,Endo vein,Chol,Kupf,Stel,Fibr,B cell,Meso
0,0.298690,0.427510,0.103279,0.041213,0.066885,0.111843,0.047357,0.003002,1.081092e-02
1,0.546834,0.002932,0.002155,0.100921,0.008674,0.045696,0.031766,0.000550,2.709512e-02
2,0.892117,0.003176,0.008136,0.176699,0.021672,0.088568,0.061912,0.001187,3.757540e-02
3,1.000000,0.002389,0.008716,0.203550,0.036523,0.178792,0.127577,0.000216,1.327807e-02
4,0.890595,0.002011,0.006584,0.173624,0.028515,0.095030,0.056764,0.000557,5.900124e-03
5,0.222120,0.098832,0.014295,0.018179,1.000000,0.107160,0.061149,0.007603,1.440479e-02
6,0.247485,0.031254,0.045740,0.017715,0.098171,1.000000,0.573760,0.001423,5.182703e-02
7,0.848275,0.002387,0.006806,0.186662,0.023811,0.217954,0.079659,0.000327,3.817444e-02
8,0.642397,0.000378,0.002993,0.141170,0.007379,0.139241,0.098599,0.000136,4.635810e-02
9,0.773819,0.004293,0.004120,0.147497,0.017846,0.098389,0.066475,0.000104,4.256161e-02


In [14]:
sc_leiden_cleanl

{'0': 0.35477935720558534,
 '1': 1.3767940063355242,
 '2': 1.3387107788059611,
 '3': 1.3235006050733258,
 '4': 1.3474122874441365,
 '5': 1.2730005924553487,
 '6': 0.541683970680054,
 '7': 1.1823370828430941,
 '8': 1.2793468245309227,
 '9': 1.3596262520712281,
 '10': 1.338315494034929,
 '11': 0.0,
 '12': 1.568746720490447,
 '13': 0.18504862253094,
 '14': 1.4882365098568995,
 '15': 0.7912719981572682,
 '16': 0.3333706821620597,
 '17': 1.5189096829978108,
 '18': 1.294733078815529,
 '19': 0.0,
 '20': 0.6551247392382691,
 '21': 0.4092226429599817,
 '22': 0.9727154332157218}